In [4]:
import pandas as pd

In [23]:
spiders = pd.read_csv(r"C:\Users\jcb260\Documents\python-assessment\spiders_data.csv")

In [17]:
spiders.head()

,speciesId,species_lsid,family,genus,species,subspecies,author,year,parentheses,distribution
0,1,urn:lsid:nmbe.ch:spidersp:000896,Actinopodidae,Actinopus,caraiba,NaN,Simon,1889,1,Venezuela
1,2,urn:lsid:nmbe.ch:spidersp:000898,Actinopodidae,Actinopus,crassipes,NaN,Keyserling,1891,1,"Brazil, Paraguay, Argentina"
2,3,urn:lsid:nmbe.ch:spidersp:000899,Actinopodidae,Actinopus,cucutaensis,NaN,Mello-Leitão,1941,0,"Colombia, Venezuela, Brazil"
3,4,urn:lsid:nmbe.ch:spidersp:000900,Actinopodidae,Actinopus,dubiomaculatus,NaN,Mello-Leitão,1923,0,Brazil
4,5,urn:lsid:nmbe.ch:spidersp:000901,Actinopodidae,Actinopus,echinus,NaN,Mello-Leitão,1949,0,Brazil


In [19]:
import statsmodels.api as sm

In [22]:
gaus = sm.GLM(spiders, , family=sm.families.Gaussian())

AttributeError: 'DataFrame' object has no attribute 'endog'

# spiders
things to possibly predict:
make sure there are graphs
make sure there is enough to show the different things we learned 
class prediction - spider family - is this predicted by species number
class prediction - distribution - is this predicted by species number
glm - is species number predictable in a glm using family + distribution